In [1]:
import pandas as pd

<font size=4 color='4DA1A9'>The outline of these notebooks were influenced by Phil Schmid's [blog post](https://www.philschmid.de/fine-tune-modern-bert-in-2025) </font>

In [1]:
# !pip install git+https://github.com/huggingface/transformers.git

<font size=4> Loading The Data </font>

In [2]:
column_names=[
        'id',                # Column 1: the ID of the statement ([ID].json).
        'label',             # Column 2: the label.
        'statement',         # Column 3: the statement.
        'subjects',          # Column 4: the subject(s).
        'speaker',           # Column 5: the speaker.
        'speaker_job_title', # Column 6: the speaker's job title.
        'state_info',        # Column 7: the state info.
        'party_affiliation', # Column 8: the party affiliation.
        
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.

        'context' # Column 14: the context (venue / location of the speech or statement).
]

train_data = pd.read_csv('./liar_dataset/train.tsv', sep='\t', header=None, names=column_names)
test_data  = pd.read_csv('./liar_dataset/test.tsv',  sep='\t', header=None, names=column_names)
valid_data = pd.read_csv('./liar_dataset/valid.tsv', sep='\t', header=None, names=column_names)

<font size=4> Preprocessing the Data </font>

In [3]:
from datasets import Dataset

def preprocess_data(data:pd.DataFrame, six_way:bool=True, metadata:bool=True):

    # these data are not usable
    data.drop(columns=[f'count_{i+1}' for i in range(5)], inplace=True)

    # encoding output labels 
    if six_way:
        numerical={'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
        data['label'] = data['label'].map(numerical)
    else:
        true_labels= ['true', 'mostly-true', 'half-true']
        data['label'] = data['label'].apply(lambda x: 1 if x in true_labels else 0)

    # fill missing columns
    data.fillna('',inplace=True)

    # adding metadata
    if metadata:
        data['statement'] = data['statement'] + ' ' + data['speaker'] + ' ' + data['speaker_job_title'] + ' ' + data['state_info'] + ' ' + data['party_affiliation'] + ' ' + data['context']

    # dropping every column other than label and statement
    data.drop(columns=data.columns.difference(['label', 'statement']), inplace=True)

    # data_dict= data.to_dict(orient='records')
    # return data_dict
    return Dataset.from_pandas(data)


In [ ]:
train_data= preprocess_data(train_data)
test_data= preprocess_data(test_data)
valid_data= preprocess_data(valid_data)

<font size=4> I like it better when it is a single dataset </font>

In [5]:
from datasets import dataset_dict

dataset_dict= dataset_dict.DatasetDict()

dataset_dict['train']= train_data
dataset_dict['test']= test_data
dataset_dict['valid']= valid_data

In [6]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'statement'],
        num_rows: 10240
    })
    test: Dataset({
        features: ['label', 'statement'],
        num_rows: 1267
    })
    valid: Dataset({
        features: ['label', 'statement'],
        num_rows: 1284
    })
})

### Tokenizer

In [7]:
from transformers import AutoTokenizer, DebertaV2Tokenizer
import torch

model_id = "microsoft/deberta-v3-base"
# model_id = "google-bert/bert-base-uncased"
# model_id='distilbert/distilbert-base-uncased'
# model_id = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length=512


# Tokenize function
def tokenizer_helper(sample):
    return tokenizer(sample['statement'], padding='max_length', truncation=True, return_tensors="pt")

/cta/users/kguray/.venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
# Tokenize the data

tokenized_dataset= dataset_dict.map(tokenizer_helper, batched=True, remove_columns=['statement'])

Map:   0%|          | 0/10240 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10240
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1267
    })
    valid: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1284
    })
})

### Classifier

In [ ]:
from transformers import AutoModelForSequenceClassification

# model_id = "microsoft/deberta-v3-base"
# model_id = "google-bert/bert-base-uncased"
# model_id='distilbert/distilbert-base-uncased'


labels_2_id={'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
id_2_labels={v:k for k,v in labels_2_id.items()}
# id_2_labels={0:'false',1:'true'}
# labels_2_id={v:k for k,v in id_2_labels.items()}

model=AutoModelForSequenceClassification.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, label2id=labels_2_id, id2label=id_2_labels, num_labels=len(id_2_labels)
)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Setting the model to run on device

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model=model.to(device)


In [12]:
import numpy as np
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
 
# Metric helper method
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [13]:
from transformers import Trainer, TrainingArguments
 
# Define training args
training_args = TrainingArguments(
    output_dir= "other_bert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    num_train_epochs=3,
    bf16=True, # bfloat16 training 
    optim="adamw_torch_fused", # improved optimizer 
    weight_decay=0.01,
    warmup_ratio=0.1,
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)
 
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.686300,0.712296,0.520249,0.356072,0.270659,0.520249
2,0.689100,0.692874,0.520249,0.356072,0.270659,0.520249
3,0.683800,0.694399,0.520249,0.356072,0.270659,0.520249


/cta/users/kguray/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/cta/users/kguray/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/cta/users/kguray/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1920, training_loss=0.686629002292951, metrics={'train_runtime': 618.2168, 'train_samples_per_second': 49.691, 'train_steps_per_second': 3.106, 'total_flos': 8082916575805440.0, 'train_loss': 0.686629002292951, 'epoch': 3.0})

In [43]:
trainer.evaluate(tokenized_dataset['test'])


{'eval_loss': 0.6761326193809509,
 'eval_accuracy': 0.659037095501184,
 'eval_f1': 0.6547883036136438,
 'eval_precision': 0.6559005596062724,
 'eval_recall': 0.659037095501184,
 'eval_runtime': 4.7314,
 'eval_samples_per_second': 267.788,
 'eval_steps_per_second': 16.908,
 'epoch': 3.0}